In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 데이터 전처리
- kaggle dataset 

In [2]:
book_df=pd.read_csv('C:\Workspaces\Bookitoki\soovin\data/book_review_data.csv')

In [3]:
# 1) 필요한 컬럼 도출
# 2) 형변환 (float > int)

book_df=book_df[['user_id',"age","rating","book_title",'book_author','year_of_publication','img_m','Summary','Category','country','Language']]
book_df[['age',"year_of_publication"]]=book_df[['age',"year_of_publication"]].astype(int)

In [4]:
# 최소 평가 개수 ( 평가가 30개보다 많은 책만 추출 )
counts=book_df['book_title'].value_counts()
title = counts[counts > 30].index
book_filter_df=book_df[book_df['book_title'].isin(title)]


In [5]:
# 최소 평가 개수 ( 평가가 20개보다 많은 사용자만 추출 )
counts=book_filter_df["user_id"].value_counts()
user_id_=counts[counts>10].index
book_filter_df = book_filter_df[book_filter_df['user_id'].isin(user_id_)]

In [6]:
# 무의미한 0점 평가 제거
rating_zero=book_filter_df[book_filter_df['rating']==0].index
book_filter_df.drop(rating_zero, axis=0, inplace=True)

In [7]:
# 한 카테고리안에 책이 5권 미만인 경우 제거
counts=book_filter_df["Category"].value_counts()
categories = counts[counts > 5].index
book_filter_df = book_filter_df[book_filter_df['Category'].isin(categories)]

idx=book_filter_df[book_filter_df["Summary"]=='9'].index
book_filter_df.drop(idx,axis=0,inplace=True)

book_filter_df["book_title"]=book_filter_df['book_title'].str.lower()
book_filter_df["Summary"]=book_filter_df["Summary"].str.lower()

In [33]:
book_filter_df.shape
len(book_filter_df['book_title'].unique())

(55349, 12)


3450

#### user_book_rating dataframe

In [9]:
user_book_df=book_filter_df.pivot_table(index='user_id',columns='book_title',values='rating',fill_value=0)
book_user_df=user_book_df.T
user_book_df.shape # user간 유사도 구할거 아니니까

(4836, 3450)

In [10]:
user_book_df.iloc[55].sort_values(ascending=False)[:10]

book_title
thinner                                                                     8.0
christine                                                                   8.0
stupid white men ...and other sorry excuses for the state of the nation!    6.0
cujo                                                                        6.0
'salem's lot                                                                0.0
sweet hereafter movie tie-in : a novel                                      0.0
sunset in st. tropez                                                        0.0
superfudge (yearling books (paperback))                                     0.0
superstitious                                                               0.0
surfacing                                                                   0.0
Name: 4334, dtype: float64

#### age_title_count dataframe

In [11]:
book_filter_df["age"].value_counts()

age
34    16829
33     1882
29     1708
28     1689
32     1585
      ...  
76        6
11        5
84        5
10        3
97        2
Name: count, Length: 78, dtype: int64

In [12]:
def make_ages(age):
    if 10<=age<20:
        return 10
    elif 20<=age<30:
        return 20
    elif 30<=age<40:
        return 30
    elif 40<=age<50:
        return 40
    elif 50<=age<60:
        return 50
    elif 60<=age<70:
        return 60
    else:
        return 70

In [13]:
book_filter_df['ages'] = book_filter_df['age'].apply(make_ages)

In [14]:
ages_book_=book_filter_df.groupby(['ages','book_title']).size().reset_index(name='count')

In [15]:
ages_book_df=ages_book_.pivot_table(index='ages',columns='book_title',values='count',fill_value=0)
book_ages_df=ages_book_df.T
book_ages_df[:100]

ages,10,20,30,40,50,60,70
book_title,,,,,,,
'salem's lot,0.0,0.0,4.0,1.0,0.0,0.0,0.0
10 lb. penalty,0.0,1.0,5.0,4.0,1.0,1.0,1.0
101 dalmatians,0.0,2.0,0.0,1.0,0.0,0.0,0.0
16 lighthouse road,1.0,1.0,6.0,3.0,2.0,0.0,0.0
1984,3.0,26.0,18.0,8.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
a stitch in time (needlecraft mysteries),0.0,1.0,4.0,1.0,3.0,0.0,0.0
a streetcar named desire,1.0,7.0,3.0,0.0,1.0,0.0,0.0
a sudden change of heart,0.0,1.0,5.0,1.0,0.0,0.0,0.0


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
ages_similarity=cosine_similarity(book_ages_df,book_ages_df)
ages_similarity_df=pd.DataFrame(ages_similarity,index=ages_book_df.columns, columns=ages_book_df.columns)

In [17]:
ages_similarity_df.shape

(3450, 3450)

#### user별 안 읽은 책 추천

In [18]:
def get_unseen_books(user_idx):
    user_read_df=user_book_df.iloc[user_idx,:]
    return user_read_df[user_read_df==0].index

In [35]:
def ages_popular_book(user_idx):
    user_ages=book_filter_df.iloc[user_idx]["ages"]
ages_popular_book(176)

# 작가 평점 추천 전처리

In [37]:
# 책의 평점으로 평균내기
bm_df = book_filter_df.groupby(['book_title', 'book_author'])['rating'].mean().reset_index()
print(bm_df.shape)
bm_df=bm_df.rename(columns={'rating':'book_mean_rating'})
bm_df

(3960, 3)


,book_title,book_author,book_mean_rating
0,'salem's lot,Stephen King,7.000000
1,10 lb. penalty,Dick Francis,7.461538
2,101 dalmatians,Dodie Smith,10.000000
3,101 dalmatians,Walt Disney,9.500000
4,16 lighthouse road,Debbie Macomber,7.076923
...,...,...,...
3955,zen and the art of motorcycle maintenance: an ...,ROBERT PIRSIG,8.129032
3956,zia,Scott O'Dell,10.000000
3957,zlata's diary: a child's life in sarajevo,Zlata Filipovic,9.333333
3958,zodiac: the eco-thriller,Neal Stephenson,7.750000


In [52]:
#책 평균 평점으로 작가 평점 대입하기
am_df = bm_df.groupby(['book_author'])['book_mean_rating'].mean().reset_index()
print(am_df.shape)
am_df = am_df.rename(columns={'book_mean_rating': 'author_mean_rating'})
am_df

(1721, 2)


,book_author,author_mean_rating
0,A. Manette Ansay,6.931818
1,A. N. Roquelaure,7.571429
2,ADRIANA TRIGIANI,7.588889
3,ALBERT CAMUS,7.400000
4,ALEXANDRA FULLER,7.952381
...,...,...
1716,Zachary Fox,5.500000
1717,Zadie Smith,7.500000
1718,Zilpha Keatley Snyder,7.111111
1719,Zlata Filipovic,9.333333


In [70]:
#작가 평점 피벗테이블
author_mean_df = bm_df.pivot_table(index='book_author', columns='book_title', values='book_mean_rating', fill_value=0)
author_mean_df.head()

book_title,'salem's lot,10 lb. penalty,101 dalmatians,16 lighthouse road,1984,1st to die: a novel,2010: odyssey two,24 hours,4 blondes,84 charing cross road,...,you just don't understand,you shall know our velocity,you'll never eat lunch in this town again,your money or your life: transforming your relationship with money and achieving financial independence,your oasis on flame lake (ballantine reader's circle),zen and the art of motorcycle maintenance: an inquiry into values,zia,zlata's diary: a child's life in sarajevo,zodiac: the eco-thriller,zoya
book_author,,,,,,,,,,,,,,,,,,,,,
A. Manette Ansay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A. N. Roquelaure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ADRIANA TRIGIANI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ALBERT CAMUS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ALEXANDRA FULLER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
author_similarity = cosine_similarity(author_mean_df)  
author_similarity_df = pd.DataFrame(author_similarity, index=author_mean_df.index, columns=author_mean_df.index)
author_similarity_df.head()

book_author,A. Manette Ansay,A. N. Roquelaure,ADRIANA TRIGIANI,ALBERT CAMUS,ALEXANDRA FULLER,ALICE STEINBACH,AMANDA QUICK,ANN BENSON,ANN BRASHARES,ANN PACKER,...,Witi Ihimaera,Wollstonecraft Mary Shelley,Yann Martel,Yxta Maya Murray,Zachary Alan Fox,Zachary Fox,Zadie Smith,Zilpha Keatley Snyder,Zlata Filipovic,Zora Neale Hurston
book_author,,,,,,,,,,,,,,,,,,,,,
A. Manette Ansay,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A. N. Roquelaure,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ADRIANA TRIGIANI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ALBERT CAMUS,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ALEXANDRA FULLER,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
def recommend_authors_books(author_name, top_n=10):
    """
    :param author_name: 추천 기준이 될 작가명
    :param top_n: 유사도가 높은 상위 작가 수
    :return: 유사 작가들의 책 리스트
    """
    # 입력한 작가가 존재하는지 확인
    if author_name not in author_similarity_df.index:
        return f"작가 '{author_name}'를 찾을 수 없습니다."

    # 유사한 작가 top_n을 기준으로 선정
    similar_authors = author_similarity_df[author_name].sort_values(ascending=False)[1:top_n+1].index

    # 추천 작가의 책 리스트 생성
    recommended_books = bm_df[bm_df['book_author'].isin(similar_authors)][['book_title', 'book_author', 'book_mean_rating']]
    return recommended_books

# 예시: 특정 작가의 유사 작가 기반 책 추천
display(recommend_authors_books('George Orwell'))

,book_title,book_author,book_mean_rating
196,all-american girl,Meg Cabot,8.333333
1365,i know why the caged bird sings,Maya Angelou,8.000000
1473,jennifer government : a novel,Max Barry,7.950000
2150,"princess in love (the princess diaries, vol. 3)",Meg Cabot,8.909091
2151,princess in the spotlight (the princess diarie...,Meg Cabot,8.529412
2417,she went all the way (avon light contemporary ...,Meggin Cabot,6.933333
2489,sloppy firsts : a novel,Megan McCafferty,8.266667
2681,temple,Matthew Reilly,6.666667
2898,the deceiver,Melanie Tem,8.000000
3033,the girls' guide to hunting and fishing,Melissa Bank,6.703297
